In [2]:
import sys
sys.executable

'/opt/anaconda3/envs/poker/bin/python'

In [3]:
import tensorflow as tf

# How to use this

Run each cell from top to bottom. 
View README.md for more infos. 

In [3]:
# Init global infos
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Activation

from tensorflow.keras.optimizers import SGD

from tensorflow.keras.utils import to_categorical

tf.keras.backend.clear_session()


In [4]:
pokerdf = pd.read_csv('train.csv',index_col = False)

In [5]:
pokercols = []
for i in range(1,6):
    pokercols.append(['S'+str(i),'C'+str(i),'cC'+str(i)])

In [6]:
pokercols = list(np.array(pokercols).flatten())

In [7]:
for i in range(1,6):
    pokerdf['cC'+str(i)] = pokerdf['C'+str(i)]

In [8]:
pokercols = pokercols + ['hand']

In [9]:
#adjusted_cols = list(pokercols[:10]) + list(pokercols[11:]) + ['hand']

In [10]:
pokerdf = pokerdf[pokercols]

In [11]:
pokerdf.head()

,S1,C1,cC1,S2,C2,cC2,S3,C3,cC3,S4,C4,cC4,S5,C5,cC5,hand
0,4,9,9,2,1,1,2,2,2,4,7,7,2,8,8,0
1,1,4,4,3,6,6,1,12,12,3,11,11,2,7,7,0
2,1,11,11,4,1,1,3,7,7,4,11,11,2,1,1,2
3,2,9,9,2,4,4,3,6,6,1,9,9,4,9,9,3
4,1,8,8,2,4,4,2,11,11,2,2,2,2,1,1,0


In [9]:
#Defining a new input for my Adam Poker application
inputs = ( 
    ("S1", ('1','2','3','4')),
    ("C1", ('1','2','3','4','5','6','7','8','9','10','11','12','13')),
    ("cC1", ("continuous",)), 
    ("S2", ('1','2','3','4')),
    ("C2", ('1','2','3','4','5','6','7','8','9','10','11','12','13')),
    ("cC2", ("continuous",)), 
    ("S3", ('1','2','3','4')),
   ("C3", ('1','2','3','4','5','6','7','8','9','10','11','12','13')),
    ("cC3", ("continuous",)), 
    ("S4", ('1','2','3','4')),
   ("C4", ('1','2','3','4','5','6','7','8','9','10','11','12','13')), 
    ("cC4", ("continuous",)), 
    ("S5", ('1','2','3','4')),
   ("C5", ('1','2','3','4','5','6','7','8','9','10','11','12','13')),
    ("cC5", ("continuous",)), 
)

In [10]:
input_shape = []
for i in inputs:
    count = len(i[1 ])
    input_shape.append(count)
input_dim = sum(input_shape)
print("input_shape:", input_shape)
print("input_dim:", input_dim)

input_shape: [4, 13, 1, 4, 13, 1, 4, 1, 4, 1, 4, 1]
input_dim: 51


In [11]:
outputs = tuple(range(0,10))  # (">50K", "<=50K")
output_dim = len(outputs)
print("output_dim:", output_dim)
print()

output_dim: 10



In [12]:
# Helpful function 2    
    
def find_means_for_continuous_types(X):
    means = []
    for col in range(len(X[0])):
        summ = 0
        count = 0.000000000000000000001
        for value in X[:, col]:
            if isinstance(value,int): 
                summ += value
                count +=1
        means.append(summ/count)
    return means

## Changing dtype to float

In [13]:
pokerdf[['C1','C2','C3','C4','C5']] = pokerdf[['C1','C2','C3','C4','C5']].astype('str')
pokerdf[['S1','S2','S3','S4','S5']] = pokerdf[['S1','S2','S3','S4','S5']].astype('str')

In [14]:
hartrain = pokerdf.drop(['C3','C4','C5'],axis=1).values

In [15]:
# Helpful function 3
def prepare_data(raw_data, means):
    
    X = raw_data[:, :-1]
    y = raw_data[:, -1:]
    
    # X:
    def flatten_persons_inputs_for_model(person_inputs):
        global inputs
        global input_shape
        global input_dim
        global means
        float_inputs = []

        for i in range(len(input_shape)):
            features_of_this_type = input_shape[i]
            is_feature_continuous = features_of_this_type == 1

            if is_feature_continuous:
                mean = means[i]
                scale_factor = 1/(2*mean)
                float_inputs.append(person_inputs[i]*scale_factor)
#                 if isinstance(person_inputs[i],float):
#                     scale_factor = 1/(2*mean)  # we prefer inputs mainly scaled from -1 to 1. 
#                     float_inputs.append(float(person_inputs[i])*scale_factor)
#                 else:
#                     float_inputs.append(mean)
            else:
                for j in range(features_of_this_type):
                    feature_name = inputs[i][1][j]

                    if feature_name == person_inputs[i]:
                        float_inputs.append(1.)
                    else:
                        float_inputs.append(0)
        return float_inputs
    
    new_X = []
    for person in range(len(X)):
        formatted_X = flatten_persons_inputs_for_model(X[person])
        new_X.append(formatted_X)
    new_X = np.array(new_X)
    
    # y:
    new_y = to_categorical(y, num_classes = 10)
    
#     new_y = []
#     for i in range(len(y)):
#         if y[i] == ">50k":
#             new_y.append((1, 0))
#         else:  # y[i] == "<=50k":
#             new_y.append((0, 1))
#     new_y = np.array(new_y)
    
    return (new_X, new_y)

## Finding means for poker data below

In [16]:
# hartrain = pokerdf.values (ignore for now)

In [17]:
means = find_means_for_continuous_types(hartrain[:,:-1])
print("Mean values for data types (if continuous):", means)

Mean values for data types (if continuous): [0.0, 0.0, 6.995241903238704, 0.0, 0.0, 7.014194322271091, 0.0, 7.014154338264694, 0.0, 6.9424630147940825, 0.0, 6.962734906037585]


In [19]:
X_train, y_train = prepare_data(hartrain, means)

In [20]:
# Explanation on data format
print("Training data format example:")
print(X_train[4], len(X_train[4]))  # 4 is a random person, from cuba.

Training data format example:
[1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         0.         0.57181725
 0.         1.         0.         0.         0.         0.
 0.         1.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.2851361
 0.         1.         0.         0.         0.78412874 0.
 1.         0.         0.         0.1440411  0.         1.
 0.         0.         0.07181086] 51


In [21]:
# Init model

mid_dim = 100

# sec_dim = 1000

model = Sequential()

model.add(Dense(mid_dim, activation='relu'))
# model.add(Dense(sec_dim, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'] )

#model.summary()

In [23]:
# Train the model

print("(training_datas, dimension):", X_train.shape)

(training_datas, dimension): (25010, 51)


In [24]:
# model.fit(new_X_train, y_train, nb_epoch=3, batch_size=16, show_accuracy=True, verbose=2)
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

313/313 [==============================] - 1s 3ms/step - loss: 1.0808 - accuracy: 0.4760 - val_loss: 0.9890 - val_accuracy: 0.5004
Epoch 2/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9868 - accuracy: 0.4994 - val_loss: 0.9876 - val_accuracy: 0.5144
Epoch 3/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9800 - accuracy: 0.5080 - val_loss: 0.9861 - val_accuracy: 0.5046
Epoch 4/5
313/313 [==============================] - 1s 3ms/step - loss: 0.9742 - acc

## New stuff here

In [25]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [26]:
from tensorflow.keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    decay_rate = 0.85
    decay_step = 1
    if epoch % decay_step == 0 and epoch:
        return lr * pow(decay_rate, np.floor(epoch / decay_step))
    return lr

In [27]:
callbacks = [LearningRateScheduler(lr_scheduler, verbose=1)]

In [39]:
loss = CategoricalCrossentropy(label_smoothing=0.1)

In [40]:
optim = Adam(learning_rate=0.005)

In [30]:
MEMORY_LIMIT = 1024
gpus = tf.config.experimental.list_physical_devices('GPU')

In [31]:
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

Virtual devices cannot be modified after being initialized


In [32]:
input_dim

51

In [43]:
mid_dim = 200

sec_dim = 100

third_dim = 100

model = Sequential()

model.add(Dense(mid_dim, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(sec_dim, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(third_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim, activation='softmax'))

model.compile(loss=loss, optimizer='Adam',metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 200)               10400     
_________________________________________________________________
dropout_9 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)               

In [44]:
model.fit(X_train, y_train,epochs=20, validation_split=0.2, batch_size=128, verbose=1)

Epoch 1/20
157/157 [==============================] - 1s 4ms/step - loss: 1.3564 - accuracy: 0.4745 - val_loss: 1.2826 - val_accuracy: 0.5014
Epoch 2/20
157/157 [==============================] - 1s 3ms/step - loss: 1.2980 - accuracy: 0.4911 - val_loss: 1.2797 - val_accuracy: 0.5052
Epoch 3/20
157/157 [==============================] - 1s 4ms/step - loss: 1.2904 - accuracy: 0.4983 - val_loss: 1.2782 - val_accuracy: 0.5124
Epoch 4/20
157/157 [==============================] - 1s 3ms/step - loss: 1.2816 - accuracy: 0.5089 - val_loss: 1.2726 - val_accuracy: 0.5222
Epoch 5/20
157/157 [==============================] - 1s 3ms/step - loss: 1.2690 - accuracy: 0.5388 - val_loss: 1.2660 - val_accuracy: 0.5396
Epoch 6/20
157/157 [==============================] - 1s 4ms/step - loss: 1.2547 - accuracy: 0.5562 - val_loss: 1.2601 - val_accuracy: 0.5478
Epoch 7/20
157/157 [==============================] - 1s 4ms/step - loss: 1.2405 - accuracy: 0.5721 - val_loss: 1.2533 - val_accuracy: 0.5540
Epoch 

In [ ]:
testdf = pd.read_csv('test.csv',index_col = False); del testdf['id']

In [ ]:
testdf[['C1','C2','C3','C4','C5']] = testdf[['C1','C2','C3','C4','C5']].astype('float64')
testdf[['S1','S2','S3','S4','S5']] = testdf[['S1','S2','S3','S4','S5']].astype('str')

In [ ]:
hartest = testdf.values

In [ ]:
testdf['ycol'] = 0

In [ ]:
X_test,_ = prepare_data(hartest,means)

In [ ]:
X_test[0]

In [ ]:
hands = model.predict_classes(X_test)

In [ ]:
hands = pd.Series(hands)

In [ ]:
hands.value_counts()

In [ ]:
submission = pd.read_csv('sampleSubmission.csv', index_col = False)

In [ ]:
submission.hand = hands

In [ ]:
submission.to_csv('baselineMLP.csv',index=False)